# RBM models
An example of a simple SEIR model written on kappa;

In [ ]:
!cat ../cv19gm/kappa/SEIR-base.xka

### Pandemic-Suite
Using common configuration files for pandemic models, we can simulate similar models using ODEs or RBM aproach.

In [1]:
cfg = '../cfg-files/SEIR-by_age.toml'
!cat {cfg}

# Pandemic Suite Configuration File		

title = 'Example of a SEIR Configuration File'
date = '2021-04-20'
user = 'Samuel'

[model]
	name = "SEIR-byage"
	compartments  = ['S','E','I','R'] 
	type = "RBM"
	

[data]
	# Real data will set initial conditions and will be available to be plotted and to calculate errors	
	datafile = false # File path for importing data from file 
	importdata = false # Import data from external source

	# parameters to use when importing from external sources 
	initdate = ''
	country = ''
	state = ''
	county = ''#'13101'
	healthservice = ''
	loc_name= '' # optional 


[parameters]
	[parameters.static]
	t_init = 0 # Initial day
	t_end = 500 # Final day
	timestep = 0.01 # Timestep for solver
	
	# Saturation dynamic
	k_I = 0
	k_R = 0

	seroprevfactor = 1 # Fraction of the total population that take part on the dynamic at the beginning
	expinfection = 0 # Factor in which exposed infect, default is 0

	mu = 0.5 # E(0) = mu*I(0)
	
	# Subreport
	pI_det = 1		
	
	[parame

In [2]:
import sys
#print(sys.path)
#sys.path.clear()
if('../cv19gm/' not in sys.path):
    sys.path.append('../cv19gm/')
import RBM

model = RBM.SEIR(cfg)



In [3]:

result = model.kappa_sim(True)
#model.integrate()
result

{'-i': '../cv19gm/kappa/SEIR-by_age.xka', '-r': '1', '-t': '500', '-p': '500', '--verbose': '1'}

c++: cout verbose
c++: clog verbose
Temp-file was already closed.


In [4]:
result.collectHistogram()
avg,sd = result.getAvgTrajectory()
print(result.listTabs())
avg.asDataFrame()

Average Trajectory
SD
Trajectory


,Susceptible,Exposed,Infected,Removed,Daily Exposed,Daily Infected,Daily Removed,Daily Inmunized
0.0,9985.0,5.0,10.0,0.0,0.0,0.0,0.0,0.0
1.0,9981.0,9.0,8.0,2.0,4.0,0.0,2.0,0.0
2.0,9979.0,7.0,12.0,2.0,6.0,4.0,2.0,0.0
3.0,9975.0,10.0,11.0,4.0,10.0,5.0,4.0,0.0
4.0,9974.0,9.0,9.0,8.0,11.0,7.0,8.0,0.0
...,...,...,...,...,...,...,...,...
219.0,1193.0,1.0,1.0,8805.0,8792.0,8796.0,8805.0,0.0
220.0,1193.0,1.0,1.0,8805.0,8792.0,8796.0,8805.0,0.0
221.0,1193.0,1.0,1.0,8805.0,8792.0,8796.0,8805.0,0.0
222.0,1193.0,1.0,0.0,8806.0,8792.0,8796.0,8806.0,0.0


In [5]:
hist_df = result.getTab('Sim[1] - Histogram of R0 (15 < age < 45)',0).asDataFrame()
hist_df.T.plot()
hist_df

IndexError: Data Table Sim[1] - Histogram of R0 (15 < age < 45)[0]

In [ ]:
sim_hists = []
sim_hists.append(result.getTab('Sim[1] - Histogram of R0 (age < 15)',0).asDataFrame())
sim_hists.append(result.getTab('Sim[1] - Histogram of R0 (15 < age < 45)',0).asDataFrame())
sim_hists.append(result.getTab('Sim[1] - Histogram of R0 (45 < age)',0).asDataFrame())
for hist in sim_hists:
    print(hist['Average'].values)

In [ ]:
import matplotlib.pyplot as plt
#plt.plot(model.S)
#plt.plot(model.E)
avg2 = avg.asDataFrame()
del avg2['Susceptible']
del avg2['Vacc']
plt = avg2.plot(yerr = sd.asDataFrame())
#plt.plot(sd.asDataFrame()['Exposed'])

In [ ]:
result.collectRawData()

In [ ]:
raw = result.getRawData('R0 (age < 15)',1)
raw

In [ ]:
import sys,os,os.path
#sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
%env LD_LIBRARY_PATH=/home/naxo/git/KaXim/Lib
!echo $LD_LIBRARY_PATH
import sys
#print(sys.path)
#sys.path.clear()
if('../cv19gm/models' not in sys.path):
    sys.path.append('../cv19gm/models')
import RBM_seir

model = RBM_seir.RBM_SEIR('../config_files/SEIRHVD-k.toml')
result = model.kappa_sim(True,time = 100,runs = 1)
#model.integrate()
result

In [ ]:
avg,sd = result.getAvgTrajectory()
model.avg[model.STATES_NAMES+["Dead","Vaccinated"]].plot(yerr = 1*sd.asDataFrame()[model.STATES_NAMES+["Dead","Vaccinated"]],
                                         ylabel = "Population",xlabel = "Time (days)")

In [ ]:
#from cv19gm.models.seirhvd import SEIRHVD
from seirhvd import SEIRHVD
model_ode = SEIRHVD(config = '../config_files/SEIRHVD.toml')
model_ode.solve()
#model_ode.results[list(model.STATES)].plot(ylabel = "Population",xlabel = "Time (days)")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(model_ode.t,model_ode.S,label='S')
plt.plot(model_ode.t,model_ode.E,label='E')
plt.plot(model_ode.t,model_ode.I,label='I')
plt.plot(model_ode.t,model_ode.R,label='R')
plt.plot(model_ode.t,model_ode.H,label='H')
plt.plot(model_ode.t,model_ode.Iv,label='Iv')
plt.plot(model_ode.t,model_ode.D,label='D')
plt.xlim(0,200)
plt.legend(loc=0)
plt.title('SEIR from SEIRHVD lib')
plt.show()

In [ ]:
#print(model_ode.sol.y.transpose()[4])
#print(model_ode.sol.y.transpose()[200])
import pandas as pd
tab = pd.DataFrame(model_ode.sol.y.transpose()[0:200],columns = model_ode.sol_cols)
pd.set_option('display.max_rows', 201)
tab

In [ ]:
avg.asDataFrame()

In [ ]:
tab = result.getTab("Sim[0] - Trajectory",0)
tab.asDataFrame()